# Воркшоп по Kafka и Spark Structured Streaming

Есть бэкенд система, которая обрабатывает покупки. Эта система в режиме реального времени отправляет данные в топик Kafka `yp.workshop.kafka.retail_data` в формате JSON. Например:

```json
{
   "InvoiceNo":"536365",
   "StockCode":"85123A",
   "Description":"WHITE HANGING HEART T-LIGHT HOLDER",
   "Quantity":"6",
   "InvoiceDate":"12/1/2010 8:26",
   "UnitPrice":"2.55",
   "CustomerID":"17850",
   "Country":"United Kingdom"
}
```

Есть другая бэкенд система, которая обрабатывает данные пользователей. Эта система также в режиме реального времени отправляет данные в топик Kafka `yp.workshop.kafka.customer_data` в формате JSON. Например:

```json
{
   "CustomerID":"12346",
   "Address":"Unit 1047 Box 4089\nDPO AA 57348",
   "Birthdate":"1994-02-20 00:46:27",
   "Email":"cooperalexis@hotmail.com",
   "Name":"Lindsay Cowan",
   "Username":"valenciajennifer"
}
```

Данные и в том, и в другом случае отправляются при **изменении или создании**.

Есть запрос создать страницу в личном кабинете каждого клиента, где бы отображалась вся история его покупок. Допустим, что эти данные будут поступать на фронтенд через API.

Задача - написать пайплайн, который в потоковом режиме будет преобразовывать сообщения о покупках таким образом, чтобы API смог забрать данные по каждому клиенту с актуальным списком покупок. 

Базовый стек: 
- Spark Structured Streaming
- MongoDB

Можно добавить любую другую технологию или базу данных, если это необходимо. Цель - сделать так, чтобы данные о покупке поступали как можно скорее на API.

## Задание 0

Все логи по умолчанию пишутся в консоль. Чтобы увидеть их в ноутбуке, необходимо выполнить следующие действия:
 - В консоли докера с `pyspark` выполнить команду `ipython profile create`;
 - В файле `.ipython/profile_default/ipython_kernel_config.py` раскомментировать строку `c.IPKernelApp.capture_fd_output = True`;
 - Перезапустить `kernel` в ноутбуке.

## Задание 1

Спроектировать пайплан. Можно нарисовать схему с базами данных, топиками Kafka и процессами Spark. Также можно опустить часть того, каким образом данные отправляются через API на фронтенд - это сейчас не так важно. 

## Задание 2

Подключиться к топику с помощью `Spark DataFrameStreamReader`

In [ ]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.streaming.listener import StreamingListener

In [ ]:
# Создать сессию Spark
spark = """ НАПИШИТЕ ВАШ КОД ЗДЕСЬ """

Настройка `ReadStream`:

In [ ]:
kafka_user = 'de-student'
kafka_pass = 'ltcneltyn'

In [ ]:
topic_name_retail = 'yp.workshop.kafka.retail_data'

df_retail = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', 'rc1b-2erh7b35n4j4v869.mdb.yandexcloud.net:9091') \
    .option('kafka.security.protocol', 'SASL_SSL') \
    .option('kafka.sasl.jaas.config', f'org.apache.kafka.common.security.scram.ScramLoginModule required username="{<Ваш код>}" password="{<Ваш код>}";') \
    .option('kafka.partition.assignment.strategy', 'org.apache.kafka.clients.consumer.RoundRobinAssignor') \
    .option('kafka.sasl.mechanism', 'SCRAM-SHA-512') \
    .option('kafka.ssl.truststore.location', '/usr/lib/jvm/java-17-openjdk-amd64/lib/security/cacerts') \
    .option('kafka.ssl.truststore.password', 'changeit') \
    .option('maxOffsetsPerTrigger', <Ваш код>) \
    .option('subscribe', <Ваш код>) \
    .option("startingOffsets", <Ваш код>) \
    .load()

In [7]:
topic_name_customer = 'yp.workshop.kafka.customer_data'

df_customer = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', 'rc1b-2erh7b35n4j4v869.mdb.yandexcloud.net:9091') \
    .option('kafka.security.protocol', 'SASL_SSL') \
    .option('kafka.sasl.jaas.config', f'org.apache.kafka.common.security.scram.ScramLoginModule required username="{<Ваш код>}" password="{<Ваш код>}";') \
    .option('kafka.partition.assignment.strategy', 'org.apache.kafka.clients.consumer.RoundRobinAssignor') \
    .option('kafka.sasl.mechanism', 'SCRAM-SHA-512') \
    .option('kafka.ssl.truststore.location', '/usr/lib/jvm/java-17-openjdk-amd64/lib/security/cacerts') \
    .option('kafka.ssl.truststore.password', 'changeit') \
    .option('maxOffsetsPerTrigger', <Ваш код>) \
    .option('subscribe', <Ваш код>) \
    .option("startingOffsets", <Ваш код>) \
    .load()

Проверяем загрузку данных:

In [ ]:
sampleQuery = df_retail.selectExpr("CAST(value AS STRING)").writeStream.format("console").start()
sampleQuery.awaitTermination(5)
sampleQuery.stop()

In [ ]:
sampleQuery = df_customer.selectExpr("CAST(value AS STRING)").writeStream.format("console").start()
sampleQuery.awaitTermination(7)
sampleQuery.stop()

## Задание 3

Написать непосредственно преобразование данных. Это преобразование будет выполняться в функции `foreachBatch`:
  - Парсинг JSON. Для этого необходима схема сообщения во формате `StructType`;
  - Фильтрация, group by, сортировка;
  - Запись в базу данных, файл;
  - ...
  
Также необходимо выбрать один из триггеров: https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.streaming.DataStreamWriter.trigger.html

In [ ]:
# Схема данных retail
retail_schema = StructType([ \
    """ НАПИШИТЕ ВАШ КОД ЗДЕСЬ """
])

In [17]:
# Схема данных customer
customer_schema = StructType([ \
   """ НАПИШИТЕ ВАШ КОД ЗДЕСЬ """                              
])

In [10]:
# Не забудьте создать базу данных и коллекции через MongoDB Compass или другой клиент
mongo_config = {
    "connection.uri": "mongodb://mongodb:27017/",
    "database": "my_database"
}

In [ ]:
# Функция, которая будет выполняться в forEachBatch
def process_retail_data(batch_df, batch_id):
    print(batch_df.count())
    """ 
      Написать логику здесь:
          1. Десереализация столбца value
          2. Парсинг строк JSON в схему Spark
          3. Группируем строки по CustomerID, Country
          4. Аггрегация, где собираем все покупки одного клиента в один список
    """
    res = batch_df \
        """ НАПИШИТЕ ВАШ КОД ЗДЕСЬ """
    
    
    # Запись в Mongo с помощью MongoSpark
    res.write \
      .format("mongodb") \
      .mode("append") \
      .option("collection", "retail_data") \
      .options(**mongo_config) \
      .save()


In [27]:
# Функция, которая будет выполняться в forEachBatch
# Для Customer просто пишем строки в коллекцию customer_data
def process_customer_data(batch_df, batch_id):
    print(batch_df.count())
    """ 
      Написать логику здесь:
          1. Десереализация столбца value
          2. Парсинг строк JSON в схему Spark
    """
    res = batch_df \
        """ НАПИШИТЕ ВАШ КОД ЗДЕСЬ """
    
    # Запись в Mongo с помощью MongoSpark
    res.write \
      .format("mongodb") \
      .mode("append") \
      .option("collection", "customer_data") \
      .options(**mongo_config) \
      .save()


In [ ]:
""" 
  Непосредственно обработка потока данных:
    1. Определяем папку checkpoints, куда Spark будет записывать свой прогреcc
    2. Добавляем функцию в foreachBatch
"""
retail_query = df_retail \
  .writeStream \
  .option("checkpointLocation", "file:///home/jovyan/checkpoints/retail_query") \
  .foreachBatch(""" НАПИШИТЕ ВАШ КОД ЗДЕСЬ """) \
  .start()

In [ ]:
""" 
То же самое для customer
"""
customer_query = df_customer \
  .writeStream \
  .option("checkpointLocation", "file:///home/jovyan/checkpoints/customer_query") \
  .foreachBatch(""" НАПИШИТЕ ВАШ КОД ЗДЕСЬ """) \
  .start()

In [ ]:
# Остановить обработку retail:
retail_query.stop()

In [ ]:
# Остановить обработку customer:
customer_query.stop()